In [42]:
import random
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

import os
from sage.all import *

FLAG = b"crypto{??????????????????????????????????????}"

def gen_keypair(G, p):
    n = random.randint(1, (p-1))
    P = n*G
    return n, P

def gen_shared_secret(P, n):
    S = P*n
    return S.xy()[0]

def encrypt_flag(shared_secret: int):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Encrypt flag
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(FLAG, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_flag'] = ciphertext.hex()
    return data

# Define Curve params
# p = 1331169830894825846283645180581
# a = -35
# b = 98
# E = EllipticCurve(GF(p), [a,b])
# G = E.gens()[0]

# # Generate keypair
# n_a, P1 = gen_keypair(G, p)
# n_b, P2 = gen_keypair(G, p)

# # Calculate shared secret
# S1 = gen_shared_secret(P1, n_b)
# S2 = gen_shared_secret(P2, n_a)

# # Check protocol works
# assert S1 == S2

# flag = encrypt_flag(S1)

# print(f"Generator: {G}")
# print(f"Alice Public key: {P1}")
# print(f"Bob Public key: {P2}")
# print(f"Encrypted flag: {flag}")

In [43]:
# ========================== output.txt ===========================
# Generator: (479691812266187139164535778017 : 568535594075310466177352868412 : 1)
# Alice Public key: (1110072782478160369250829345256 : 800079550745409318906383650948 : 1)
# Bob Public key: (1290982289093010194550717223760 : 762857612860564354370535420319 : 1)
# Encrypted flag: {'iv': 'eac58c26203c04f68d63dc2c58d79aca', 'encrypted_flag': 'bb9ecbd3662d0671fd222ccb07e27b5500f304e3621a6f8e9c815bc8e4e6ee6ebc718ce9ca115cb4e41acb90dbcabb0d'}

In [44]:
# ======================= decrypt.py =========================
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')

In [45]:
# ================================ SOLVE ============================
import random
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

import os
from sage.all import *

# Đề bài:
p = 1331169830894825846283645180581
a = -35
b = 98
E = EllipticCurve(GF(p), [a,b])

G = E(479691812266187139164535778017, 568535594075310466177352868412)
assert G == E.gens()[0]

A = E(1110072782478160369250829345256, 800079550745409318906383650948)
B = E(1290982289093010194550717223760, 762857612860564354370535420319)

data = {'iv': 'eac58c26203c04f68d63dc2c58d79aca', 'encrypted_flag': 'bb9ecbd3662d0671fd222ccb07e27b5500f304e3621a6f8e9c815bc8e4e6ee6ebc718ce9ca115cb4e41acb90dbcabb0d'}

# Solution:
# nA = discrete_log(B, G, operation="+")
# print(nA)

# share_key, _ = (nA * B).xy()
# print(share_key)

# print(decrypt_flag(share_key, data['iv'], data['encrypted_flag']))

In [46]:
# Solution: MOV attack
# Base on: https://github.com/jvdsn/crypto-attacks/blob/master/attacks/ecc/mov_attack.py
def mov_attack(E, G, R, k_max=6, max_tries = 10):
    """
    Solves the discrete logarithm problem using the MOV attack.
    More information: Harasawa R. et al., "Comparing the MOV and FR Reductions in Elliptic Curve Cryptography" (Section 2)
    :param E: Elliptic curve
    :param G: the base point
    :param R: the point multiplication result
    :param k: the embedding degree
    :param max_tries: the maximum amount of times to try to find l (default: 10)
    :return: l such that l * G == R, or None if l was not found
    """
    
    q = G.order()
    p = E.base_ring().order() # Số nguyên tố p là kích thước ECC
    
    k = 0
    for i in range(1, k_max+1, 1):
        if (pow(p, i, q) == 1):
            k = i 
            break
    print(f"Embedding degree = {k}")
    
    Ek = E.base_extend(GF(p**k))      # Ek mở rộng, cùng a, b nhưng trên trường GF(p^k)

    Gk = Ek(G)
    Rk = Ek(R)

    # Công thức tính nhanh logarit theo weil
    for i in range(max_tries):
        Q_ = Ek.random_point()
        m = Q_.order()
        d = gcd(m, q)
        Q = (m // d) * Q_
        if Q.order() != q:
            continue

        if (alpha := Gk.weil_pairing(Q, q)) == 1:
            continue

        beta = Rk.weil_pairing(Q, q)
        l = beta.log(alpha)
        return int(l)

    return None
    
nA = mov_attack(E, G, A)

share_key, _ = (nA * B).xy()
print(share_key)
print(decrypt_flag(share_key, data['iv'], data['encrypted_flag']))

Embedding degree = 2
57514367079882430785803122958
crypto{MOV_attack_on_non_supersingular_curves}
